In [1]:
import os
import string
import annoy
import codecs
from random import sample 
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

##### Осуществлён препроцессинг текста (как минимум удаление знаков препинания, приведение к нижнему регистру, стемминг/лемматизация).

In [2]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [3]:
product_df = pd.read_csv('ProductsDataset.csv')

In [4]:
product_list = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

product_df['title'].dropna().apply(lambda row: product_list.append(preprocess_txt(str(row))))
w2v_pr_list= product_list 
product_df['descrirption'].dropna().apply(lambda row: product_list.append(preprocess_txt(str(row))))
len(product_list)

69085

In [5]:
question = None
written = False

with codecs.open("prepared_answers.txt","w", "utf-8") as fout:
    with codecs.open("Otvety.txt", "r", "utf-8") as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

In [6]:
sentences = []
c = 0

with codecs.open("Otvety.txt", "r", "utf-8") as fin:
    for line in tqdm(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

0it [00:00, ?it/s]

In [7]:
sentences = [i for i in sentences if len(i) > 2]
IR_df = pd.DataFrame({'data': product_list})
concat_df = pd.DataFrame({'data': sample(sentences,70000)})
IR_df['type'] = 'product'
concat_df['type'] = 'other'
IR_df = pd.concat([IR_df,concat_df])

##### Текст векторизирован любым из изученных способов (CountVectorizer, TfidfVectorizer, HashingVectorizer, Word2Vec).

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, preprocessing


x = []
y = IR_df['type']

for i in IR_df['data']:
    x.append(" ".join(i).strip())

tfidf = TfidfVectorizer(ngram_range=(1, 3)).fit(x)
x = tfidf.transform(x)


##### Выборка разделена на обучающую и валидационную

In [9]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(x, y)

### Обучить классификатор: продуктовый запрос vs. всё остальное (продуктовым можно считать запрос, который равен названию или описанию товара).

##### Обучен классификатор с расчётом метрик на валидации (любое семейство алгоритмов, но предпочтительнее просто логистическая регрессия).

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

intent_recognition = LogisticRegression().fit(train_x, train_y)


##### Модель сохранена и при применении загружается из pkl-файла (или аналога).

In [11]:
import pickle

pickle.dump(intent_recognition, open('intent_recognitionl.pkl', 'wb'))
intent_recognition = pickle.load(open('intent_recognitionl.pkl', 'rb'))
y_pred = intent_recognition.predict(valid_x)
accuracy_score(valid_y,y_pred)

0.9564592200621189

### Реализована болталка

Все вопросы из датасета свёрнуты Word2Vec в векторное представление.
Построен индекс по вопросам.
На запрос в болталку происходит поиск ближайшего вопроса и возвращается ответ на этот вопрос.

In [12]:
talk_w2v = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5) 
talk_w2v.save("talk_w2v")

In [13]:
talk_index = annoy.AnnoyIndex(100 ,'angular') 

talk_index_map = {}
counter = 0

with codecs.open("prepared_answers.txt", "r", "utf-8") as f:
    for line in tqdm(f):
        n_w2v = 0
        spls = line.split("\t")
        talk_index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in talk_w2v.wv:
                vector += talk_w2v.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        talk_index.add_item(counter, vector)
        counter += 1

talk_index.build(10) #строим 10 деревьев
talk_index.save('talk_speaker.ann')

0it [00:00, ?it/s]

True

### Реализован поиск похожих товаров в контентной части бота

##### Все названия товаров свёрнуты в векторное представление Word2Vec (предобученном или обученном на исходном датасете). ).

In [14]:
product_w2v = Word2Vec(sentences=w2v_pr_list, vector_size=100, min_count=1, window=3) #кол-во измерений вектора, мин кол-во раз когда вектор попадется, "окно" сочетаний со словами вокрут взятого слова
product_w2v.save("product")

##### Построен индекс по названиям документов.Для товарных запросов реализован поиск в индексе (запрос также оборачивается Word2Vec, происходит проход в индекс).


In [17]:
product_index = annoy.AnnoyIndex(100 ,'angular') 

product_index_map = {}

for i in range(len(product_df)):
    n_w2v = 0
    product_index_map[i] = [product_df['product_id'][i],product_df['title'][i],product_df['descrirption'][i]]
    product = preprocess_txt(product_df['title'][i])
    vector = np.zeros(100)
    for word in product:
        if word in product_w2v.wv:
            vector += product_w2v.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    product_index.add_item(i, vector)

product_index.build(10) #строим 10 деревьев
product_index.save('product_speaker.ann')        

True

##### Добавить логику поиска похожих товаров по продуктовому запросу.
##### Не был уверен как лучше оформить, так что просто засунул в print

In [18]:
def recommendations(index,vector,index_map):
    rec_indexes = index.get_nns_by_vector(vector, 5)
    produc_info = index_map[rec_indexes[0]]
    print('Ваш товар:\n' + produc_info[1],"\nОписание:",produc_info[2])
    print('Похожие товары:')
    for i in range(1,len(rec_indexes)):
        produc_info = index_map[rec_indexes[i]]
        print(i, produc_info[1],"\nОписание:",produc_info[2])
        

##### Вся логика должна быть завёрнута в метод get_answer(). Ответ на продуктовый запрос должен иметь вид "product_id title".

In [19]:
def get_answer(message):
    message = preprocess_txt(message)
    intent_vec = tfidf.transform([" ".join(message)])
    if intent_recognition.predict(intent_vec) == 'product':
        index_map = product_index_map
        w2v = product_w2v.wv
        index = product_index
    else:
        index_map = talk_index_map
        w2v = talk_w2v.wv
        index = talk_index
    n_w2v = 0
    vector = np.zeros(100)
    for word in message:
        if word in w2v:
            vector += w2v[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    if index_map != talk_index_map: 
        recommendations(index,vector,index_map)
        return index_map[answer_index[0]][0]+' '+index_map[answer_index[0]][1]
    else:
        return index_map[answer_index[0]]

In [20]:
get_answer('Джинсовая юбка')

Ваш товар:
Джинсовая юбка 
Описание: Продаю фирменную и очень качественную джинсовую юбку Зара. Торг.
Похожие товары:
1 Юбка джинсовая 
Описание: Юбка джинсовая ,26 размер, стретч, длина -40 см .
2 Джинсовые юбки 
Описание: 1)Мини-юбка джинсовая, Размер S. Талия-35,бедра-45,длина-41. Новая. В наличии 2шт. Цена-600р.
2) Юбка befree, р-н 36. Б/у, состояние хорошее. Цена-400р.
По договоренности возможна встреча на ст.метро Кантемировская.
3 Юбка джинсовая 
Описание: Размер 40-42
4 Юбка джинсовая 
Описание: В отличном состоянии!!!


'5940d3a7bd36c023c4153162 Джинсовая юбка'

In [21]:
get_answer('Как дела?')

'хуже уже некуда. \n'